In [1]:
import pandas as pd
import os.path
from pathlib import Path
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',100)

out_path = '/net/isilonP/public/rw/homes/uni_adm/tmp/log_parsing/2021.01.08-16.58.35'
reduce_path = os.path.join(out_path, 'reduce')
results_path = os.path.join(out_path, 'results')

Path(results_path).mkdir(exist_ok=True)

# Overview

The data presented here is from parsing the log file from 2018-01-01 to 2020-07-16. There are several filtering steps:

1. Bots/crawlers are removed by looking at each request's user agent string using [ua-parser](https://github.com/ua-parser)
2. Removing any uknown user agent strings eg `-`
3. Must be a `GET` request
4. Request must be a success (`200`)
5. Must not be a faceted query ie includes `&fil=` in the resource.

In [2]:
df_bytes = pd.read_csv(os.path.join(reduce_path, 'bytes.csv'), names=[
    'date',
    'bytes',
])
df_bytes = df_bytes.groupby('date').sum().sort_index()

In [3]:
df_bytes.index.min()

'2018-01-01'

In [4]:
df_bytes.index.max()

'2020-07-16'

# Queries by user type and namespace

In [5]:
# use nrows=1e6 to limit for testing
df = pd.read_csv(os.path.join(reduce_path, 'parsed.csv'), encoding='utf-8', names=[
    'Namespace',
    'Application',
    'Query'
])

In [6]:
def save_csv_for_all_namespaces(dfin, user_type):
    namespaces = set(dfin['Namespace'])
    for namespace in namespaces:
        csv_path = os.path.join(results_path, f'{user_type}_{namespace}.csv')
        partitioned = dfin[dfin['Namespace']==namespace]['Query'].value_counts()
        if not partitioned.empty:
            reordered = partitioned.reset_index().rename(columns={'index':'Query','Query':'Count'})[['Count', 'Query']]
            print(f'User type={user_type} Namespace={namespace}')
            print(reordered.head(100).to_string(index=False))
            print('-'*100, '\n'*2)
            reordered.to_csv(csv_path, index=False, encoding='utf-8')

### Application breakdown

In [7]:
df_counts = df['Application'].value_counts().to_frame()
df_counts['Percentage'] = df_counts['Application'] / sum(df_counts['Application'])
df_counts.style.format({
    'Percentage': '{:,.2%}'.format,
})

,Application,Percentage
Python Requests,99861490,52.66%
Chrome,21587862,11.38%
Apache-HttpClient,17345401,9.15%
Wget,10590939,5.58%
Firefox,8431076,4.45%
libwww-perl,7771644,4.10%
IE,4758045,2.51%
Bytespider,3257142,1.72%
Safari,3049134,1.61%
Edge,2854941,1.51%


Out of curiosity had a look at the Thunderbird and Facebook queries:

In [8]:
set(df[df['Application'] == 'Thunderbird']['Query'])

{'interpro ipr014000 or interpro ipr014002 or interpro ipr014003', 'tegument'}

In [9]:
set(df[df['Application'] == 'Facebook']['Query'])

{'*',
 '1.-.-.- taxonomy:acromyrmex echinatior (panamanian leafcutter ant) (acromyrmex octospinosus echinatior) [103372]',
 '10-kda',
 '1093da',
 '1avw',
 '1ema',
 '1oel',
 '1p 34.2 duplication',
 '1p34.2 duplication',
 '1q21',
 '2.4.1.19',
 '3 hydroxyanthranilate 3 4 dioxygenase',
 '3.2.1.41',
 '3faw',
 '3faw pullulanase',
 '6.4.1.1.',
 '94 aa',
 'aa 500',
 'aa 500 features',
 'aah10943',
 'ab010145',
 'abdominal aortic aneurysm',
 'acad9',
 'accession:p04637',
 'accession:p38398',
 'accession:p84243',
 'accession:q9nzc9',
 'actin',
 'adn',
 'adrenocorticotropic',
 'af013254',
 'af043303',
 'af052018 or af052017 or af052016 or af052015 or af052014 or af019908 or af134518 or af134517 or af134516 or af134515 or af134514 or af133430',
 'af304460',
 'akt',
 'alanina',
 'alanine',
 'albumin',
 'albumin rat',
 'albúmina',
 'alcohol dehydrogenase',
 'ali',
 'alk5',
 'amylase',
 'ancestor:1643685',
 'ancestor:4896',
 'ancestor:7208',
 'ancestor:8711',
 'ancestor:9844',
 'annotation:(type:ca_b

### Partition the queries
Restrict queries submitted by these applications to as they constitute the vast majority and will help filter any bots user agents that are actually bots:

In [10]:
programmatic_apps = {
    'Python Requests',
    'Wget',
    'Apache-HttpClient',
    'libwww-perl',
    'curl',
    'Java'
}
browser_apps = {
    'Chrome',
    'IE',
    'Firefox',
    'Opera',
    'Safari',
    'QQ Browser',
    'Edge',
    'Netscape',
    'Mobile Safari',
    'Sogou Explorer',
    'Chrome Mobile',
    'UC Browser',
    'Chromium',
    'Samsung Internet',
    'Chrome Mobile iOS',
}
assert not len(programmatic_apps & browser_apps)
df_browser = df[df['Application'].isin(browser_apps)]
df_programmatic = df[df['Application'].isin(programmatic_apps)]

Percentage of browser queries considered:

In [11]:
100*len(df_browser)/(len(df) - len(df_programmatic))

91.59568102533402

Percentage of programmatic queries considered:

In [12]:
100*len(df_programmatic)/(len(df) - len(df_browser))

96.95768795603531

### Top 100 browser queries for each namespace

In [13]:
save_csv_for_all_namespaces(df_browser, 'browser')

User type=browser Namespace=citations
 Count                                             Query
  4939                                                 *
   480                         title:protein interaction
   434  kuopio ischaemic heart disease risk factor study
   352                              author:oppermann f s
   344                                 author:graves g r
   312                                  author:ashburner
   307                                    published:1951
   264                              author:pierson l s 3
   260                                   author:sanger f
   138                             journal:j. exp. biol.
   127                                  author:sanger f.
   124                           author:pierson l.s. iii
   113                      citedin:(p93003) and 9330910
   112            mappedin:(uniprot:p01256) and 12030813
   104                                     finckh parkin
    80                                journal:j ex

### Programmatic library breakdown

In [14]:
df_programmatic['Application'].value_counts()

Python Requests      99861490
Apache-HttpClient    17345401
Wget                 10590939
libwww-perl           7771644
curl                  2513980
Java                      791
Name: Application, dtype: int64

### Top 100 programmatic queries for each namespace

In [15]:
save_csv_for_all_namespaces(df_programmatic, 'programmatic')

User type=programmatic Namespace=citations
 Count                                             Query
    60                  mappedin:(uniprot:(reviewed:no))
    54                                                 *
    41                                     name:analytes
    37                                       name:values
    37                                     name:cannabis
    36                                    name:indicates
    36                                      name:samples
    35                                     name:extracts
    35                                       name:ranges
    34                                       name:limits
    32                                  name:metabolites
    31                                              rpob
    31                                  name:cannabinoid
    30                                    name:compounds
    27                                         name:acid
    26                                     na